Once the data is preprocessed, the next step is to transform it into a format that data mining techniques can analyze. This step involves reducing the data dimensionality, aggregating the data, normalizing it, and discretizing it to prepare it for further analysis.

In [8]:
import utils
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import warnings 
warnings.filterwarnings('ignore')
import pickle as pkl

# Load data
df: pd.DataFrame = utils.load_original_data()


In [10]:
# standardize data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

# save data
with open('../artifacts/fetal_health_preprocessed.pkl', 'wb') as f:
    pkl.dump(df_scaled, f)

# save scaler
with open('../artifacts/scaler.pkl', 'wb') as f:
    pkl.dump(scaler, f)